## 1. 라이브러리 로드

## 1️⃣ 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
import optuna
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

c:\Users\Owner\Desktop\유틸\store-sales-time-series-forecasting\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 📚 설명
- **xgboost, lightgbm**: 그래디언트 부스팅 기반 고성능 머신러닝 모델
- **sklearn.metrics**: RMSE, MAE, MAPE 등 평가 지표 계산
- **warnings.filterwarnings**: 노트북 실행 중 경고 메시지 억제로 가독성 향상
- **matplotlib, seaborn**: 모델 성능 및 특성 중요도 시각화

## 2️⃣ 전처리된 데이터 로드

In [2]:
# 전처리된 데이터 로드 (02_Data_Preprocessing_v2.ipynb의 출력)
train_df = pd.read_csv('train_processed.csv')
test_df = pd.read_csv('test_processed.csv')

print(f"Train 데이터 형태: {train_df.shape}")
print(f"Test 데이터 형태: {test_df.shape}")
print(f"\nTrain 컬럼: {train_df.columns.tolist()}")
print(f"\nTrain 데이터 타입:\n{train_df.dtypes}")

Train 데이터 형태: (3008016, 97)
Test 데이터 형태: (28512, 96)

Train 컬럼: ['id', 'date', 'store_nbr', 'sales', 'onpromotion', 'year', 'month', 'day', 'quarter', 'dayofweek', 'week', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos', 'city', 'state', 'cluster', 'oil_price', 'is_holiday', 'n_transactions', 'sales_lag_1', 'sales_lag_7', 'sales_lag_14', 'sales_lag_30', 'n_transactions_lag_1', 'n_transactions_lag_7', 'n_transactions_lag_14', 'n_transactions_lag_30', 'sales_roll_mean_7', 'sales_roll_std_7', 'sales_roll_min_7', 'sales_roll_max_7', 'sales_roll_mean_14', 'sales_roll_std_14', 'sales_roll_min_14', 'sales_roll_max_14', 'sales_roll_mean_30', 'sales_roll_std_30', 'sales_roll_min_30', 'sales_roll_max_30', 'n_transactions_roll_mean_7', 'n_transactions_roll_std_7', 'n_transactions_roll_min_7', 'n_transactions_roll_max_7', 'n_transactions_roll_mean_14', 'n_transactions_roll_std_14', 'n_transactions_roll_min_14', 'n_t

### 📚 설명
- **train_processed.csv**: 02 노트북에서 생성한 90개+ 특성을 포함한 훈련 데이터
  - 약 3백만 행(2013-2015 매일 54개 매장)
  - sales: 예측할 목표값
  - 90+ 특성: 날짜, 시차, 롤링, YoY, 카테고리 등
- **test_processed.csv**: 2016년 데이터로, 같은 특성 구조
  - 약 2.8만 행(테스트 기간 전체)

## 3️⃣ 특성과 목표값 분리

In [18]:
# 메타데이터 컬럼 (모델 훈련에서 제외)
meta_cols = ['sales', 'store_nbr', 'date']

# Train 데이터에서 특성과 목표값 분리
X_train = train_df.drop(columns=meta_cols, errors='ignore')
y_train = train_df['sales']

# Test 데이터에서 특성 분리 (목표값 없음)
X_test = test_df.drop(columns=meta_cols, errors='ignore')

# 문자열 컬럼 제거 (XGBoost는 숫자형만 허용)
object_cols = X_train.select_dtypes(include=['object']).columns.tolist()
X_train = X_train.drop(columns=object_cols, errors='ignore')
X_test = X_test.drop(columns=object_cols, errors='ignore')

# LightGBM 호환성: 컬럼명에서 특수 문자 제거 (대괄호, 한글, 특수 문자 등)
# LightGBM은 JSON 파싱 시 특수 문자를 지원하지 않으므로 정규화 필수
def clean_column_names(df):
    """LightGBM 호환 컬럼명으로 변환"""
    new_cols = {}
    for col in df.columns:
        # 대괄호 제거 및 공백 제거
        clean_col = col.replace('[', '').replace(']', '').replace(' ', '_')
        # 한글 문자 제거
        clean_col = ''.join(c for c in clean_col if ord(c) < 128 or c == '_')
        new_cols[col] = clean_col
    return df.rename(columns=new_cols)

X_train = clean_column_names(X_train)
X_test = clean_column_names(X_test)

print(f"제거된 문자열 컬럼: {object_cols}")
print(f"Train 특성 수: {X_train.shape[1]}")
print(f"Train 샘플 수: {X_train.shape[0]}")
print(f"Train 특성 dtype:\n{X_train.dtypes.value_counts()}")

# 메타데이터 보존 (나중에 제출 형식 생성 시 필요)
# test_df에서 'family'가 없으므로, 'id'에서 추출
if 'id' in test_df.columns:
    # id 형식: store_nbr_family_date
    test_meta = test_df[['id', 'store_nbr', 'date']].copy()
    # id를 문자열로 변환 후 family 추출
    test_meta['family'] = test_meta['id'].astype(str).str.split('_').str[1]
else:
    # 원-핫 인코딩된 family 컬럼들에서 family명 복원
    family_cols = [col for col in test_df.columns if col.startswith('family_')]
    test_meta = test_df[['store_nbr', 'date']].copy()
    # 각 행에서 1인 family 찾기
    test_meta['family'] = test_df[family_cols].idxmax(axis=1).str.replace('family_', '')

print(f"\n목표값 통계:\n{y_train.describe()}")
print(f"\nTest 메타데이터 샘플:\n{test_meta.head()}")

제거된 문자열 컬럼: ['city', 'state']
Train 특성 수: 92
Train 샘플 수: 3008016
Train 특성 dtype:
float64    41
bool       38
int64      13
Name: count, dtype: int64

목표값 통계:
count    3.008016e+06
mean     3.582691e+02
std      1.103511e+03
min      0.000000e+00
25%      0.000000e+00
50%      1.100000e+01
75%      1.960000e+02
max      1.247170e+05
Name: sales, dtype: float64

Test 메타데이터 샘플:
        id  store_nbr        date  family
0  3000888          1  2017-08-16     NaN
1  3000889          1  2017-08-16     NaN
2  3000890          1  2017-08-16     NaN
3  3000891          1  2017-08-16     NaN
4  3000892          1  2017-08-16     NaN


### 📚 설명
- **메타데이터 제외**: 'sales', 'store_nbr', 'family', 'date'는 모델 입력이 아님
- **X_train**: 90개+ 특성 (날짜 기반, 시차, 롤링, YoY, 카테고리 인코딩)
- **y_train**: 판매액 (목표값, 범위 0-8,359)
- **X_test**: Train과 동일한 구조의 특성 (Train 필터링 전 80%)
- **test_meta**: 테스트 데이터의 메타데이터 (제출 파일 생성 시 필요)

## 4️⃣ 시간 기반 훈련/검증 분할

In [8]:
# 시계열 데이터는 시간 순서 유지 필수 (랜덤 셔플 금지)
split_point = int(len(X_train) * 0.8)  # 80/20 분할

X_train_split = X_train[:split_point]
y_train_split = y_train[:split_point]

X_val = X_train[split_point:]
y_val = y_train[split_point:]

print(f"훈련 세트: {X_train_split.shape[0]} 샘플")
print(f"검증 세트: {X_val.shape[0]} 샘플")
print(f"시간 기반 분할 비율: 80% (훈련) / 20% (검증)")
print(f"\n훈련 데이터 기간: 2013년 초 ~ 2015년")
print(f"검증 데이터 기간: 2015년 중반 ~ 2015년 말")

# 컬럼명 확인 (특수 문자 검사)
print(f"\n컬럼명 샘플 (처음 5개):")
for i, col in enumerate(X_train.columns[:5]):
    print(f"  {i}: {repr(col)} (타입: {type(col).__name__})")

# 특수 문자가 있는 컬럼 검사
special_char_cols = []
for col in X_train.columns:
    if any(ord(c) >= 128 for c in col):
        special_char_cols.append(col)
    elif any(c in '[](){}' for c in col):
        special_char_cols.append(col)

if special_char_cols:
    print(f"\n특수 문자/한글 포함 컬럼 ({len(special_char_cols)}개):")
    for col in special_char_cols[:10]:
        print(f"  {repr(col)}")

훈련 세트: 2406412 샘플
검증 세트: 601604 샘플
시간 기반 분할 비율: 80% (훈련) / 20% (검증)

훈련 데이터 기간: 2013년 초 ~ 2015년
검증 데이터 기간: 2015년 중반 ~ 2015년 말

컬럼명 샘플 (처음 5개):
  0: 'id' (타입: str)
  1: 'onpromotion' (타입: str)
  2: 'year' (타입: str)
  3: 'month' (타입: str)
  4: 'day' (타입: str)


## 4️⃣ HPO (Hyperparameter Optimization) 준비

In [5]:
# HPO 평가 함수 정의
def evaluate_model(y_true, y_pred):
    """RMSLE 점수 계산 (Optuna 최적화 지표)"""
    y_pred_clipped = np.maximum(y_pred, 0)
    y_true_clipped = np.maximum(y_true, 0)
    rmsle = np.sqrt(np.mean((np.log1p(y_true_clipped) - np.log1p(y_pred_clipped))**2))
    return rmsle

print("✓ HPO 준비 완료 (Optuna 라이브러리 준비됨)")
print("  - 다음 셀에서 XGBoost HPO 시작")
print("  - 시간 소요: 약 5-10분 (설정에 따라 변동)")

✓ HPO 준비 완료 (Optuna 라이브러리 준비됨)
  - 다음 셀에서 XGBoost HPO 시작
  - 시간 소요: 약 5-10분 (설정에 따라 변동)


### 📚 설명
- **시간 기반 분할**: 시계열 데이터는 과거를 사용하여 미래를 예측하므로, 시간 순서를 반드시 유지해야 함
- **80/20 분할**: 훈련 2.4M 행, 검증 0.6M 행
- **랜덤 셔플 금지**: 데이터 누수(data leakage) 방지 - 미래 데이터가 과거 예측에 영향을 주지 않도록
- **검증 세트의 역할**: 모델이 보지 못한 미래 데이터에서 성능 평가 (실제 Kaggle 테스트와 유사)

## 5️⃣ 평가 지표 함수 정의

In [11]:
def calculate_metrics(y_true, y_pred, name="Model"):
    """
    회귀 모델의 주요 평가 지표 계산
    """
    # RMSE: Root Mean Squared Error (Kaggle 주 평가 지표)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # RMSLE: Root Mean Squared Logarithmic Error
    # 큰 값에 덜 민감, 비음수만 가능하므로 클리핑 필요
    y_pred_clipped = np.maximum(y_pred, 0)
    y_true_clipped = np.maximum(y_true, 0)
    rmsle = np.sqrt(np.mean((np.log1p(y_true_clipped) - np.log1p(y_pred_clipped))**2))
    
    # MAE: Mean Absolute Error
    mae = mean_absolute_error(y_true, y_pred)
    
    # MAPE: Mean Absolute Percentage Error
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    print(f"{name} 평가 지표:")
    print(f"  RMSE:  {rmse:.4f}")
    print(f"  RMSLE: {rmsle:.4f} (Kaggle 점수)")
    print(f"  MAE:   {mae:.4f}")
    print(f"  MAPE:  {mape:.4f}")
    
    return {'RMSE': rmse, 'RMSLE': rmsle, 'MAE': mae, 'MAPE': mape}

### 📚 설명
- **RMSLE (Kaggle 평가지표)**: 큰 값의 오차보다 작은 값의 오차에 더 많은 가중치 부여
  - log 변환으로 왜곡된 분포 교정
  - 음수 예측 클리핑 필수 (log 정의역 필요)
- **RMSE**: 오차의 크기를 원래 단위로 반영
- **MAE**: 평균 절대값 오차 (이상치에 덜 민감)
- **MAPE**: 백분율로 표현된 오차 (상대적 성능 비교)

## 6️⃣ 기준선(Baseline) 모델

In [12]:
# 기준선: 훈련 데이터의 평균값 사용
baseline_pred = np.full_like(y_val, y_train_split.mean())

print(f"훈련 데이터 판매액 평균: {y_train_split.mean():.2f}")
baseline_metrics = calculate_metrics(y_val, baseline_pred, "Baseline (Mean)")

print("\n기준선의 의미: 모든 예측값이 훈련 세트의 평균")
print("→ 어떤 모델도 이것보다 좋아야 함")

훈련 데이터 판매액 평균: 294.39
Baseline (Mean) 평가 지표:
  RMSE:  1773.1370
  RMSLE: 3.7513 (Kaggle 점수)
  MAE:   699.6799
  MAPE:  394159403197302784.0000

기준선의 의미: 모든 예측값이 훈련 세트의 평균
→ 어떤 모델도 이것보다 좋아야 함


### 📚 설명
- **Baseline 모델**: 가장 간단한 모델로 성능 하한선 설정
  - 모든 예측을 훈련 평균값으로 설정
  - 모든 다른 모델은 이보다 나은 성능을 가져야 함
- **기준선의 역할**: 
  - 모델 개선도 측정 기준점
  - 특성 엔지니어링의 실제 효과 확인
  - 과도한 복잡성이 정당한지 판단

## 7️⃣ XGBoost 모델 훈련

## 7️⃣-1 XGBoost HPO (Hyperparameter Optimization)

In [13]:
# XGBoost Objective Function for Optuna (GPU 버전)
def xgb_objective(trial):
    """Optuna를 사용한 XGBoost 하이퍼파라미터 최적화 (GPU)"""
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42,
        'eval_metric': 'rmse',
        'verbose': 0,
        'device': 'cuda',
        'tree_method': 'hist'
    }
    
    try:
        model = xgb.XGBRegressor(**params)
        model.fit(
            X_train_split, y_train_split,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
        
        # 검증 세트에서 예측
        y_pred = model.predict(X_val)
        rmsle = evaluate_model(y_val, y_pred)
        
        return rmsle
    except Exception as e:
        print(f"Trial 실패: {e}")
        return float('inf')

# Optuna Study 실행
print("XGBoost HPO 시작 (GPU 가속, 약 1-2분 소요)...")
sampler = TPESampler(seed=42)
xgb_study = optuna.create_study(
    direction='minimize',
    sampler=sampler
)

# 최적화 실행 (n_trials는 시간에 따라 조정 가능)
xgb_study.optimize(xgb_objective, n_trials=30, show_progress_bar=True)

# 최적 파라미터 출력
print("\n✓ XGBoost HPO 완료!")
print(f"최적 RMSLE: {xgb_study.best_value:.6f}")
print(f"최적 하이퍼파라미터:")
for key, value in xgb_study.best_params.items():
    print(f"  {key}: {value}")

xgb_best_params = xgb_study.best_params.copy()
xgb_best_params['random_state'] = 42
xgb_best_params['eval_metric'] = 'rmse'
xgb_best_params['verbose'] = 0
xgb_best_params['device'] = 'cuda'
xgb_best_params['tree_method'] = 'hist'

[I 2025-12-16 23:44:29,996] A new study created in memory with name: no-name-dc1436c0-225a-4d49-aadb-d4eecc435c67


XGBoost HPO 시작 (GPU 가속, 약 1-2분 소요)...


Best trial: 0. Best value: 1.74358:   3%|▎         | 1/30 [00:14<06:58, 14.45s/it]

[I 2025-12-16 23:44:44,442] Trial 0 finished with value: 1.743581906393733 and parameters: {'n_estimators': 362, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'min_child_weight': 2, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 1.743581906393733.


Best trial: 1. Best value: 1.42639:   7%|▋         | 2/30 [00:37<09:12, 19.73s/it]

...

[I 2025-12-16 23:54:32,503] Trial 27 finished with value: 1.6289255371540072 and parameters: {'n_estimators': 298, 'learning_rate': 0.03680553392073916, 'max_depth': 7, 'subsample': 0.6927383269609955, 'colsample_bytree': 0.9238406487953296, 'min_child_weight': 1, 'gamma': 4.6160274505017975}. Best is trial 19 with value: 1.2590178473485025.


Best trial: 19. Best value: 1.25902:  97%|█████████▋| 29/30 [10:25<00:21, 21.83s/it]

[I 2025-12-16 23:54:55,841] Trial 28 finished with value: 1.3202992519296053 and parameters: {'n_estimators': 491, 'learning_rate': 0.05955835244995433, 'max_depth': 9, 'subsample': 0.5804711806778007, 'colsample_bytree': 0.8633179964525198, 'min_child_weight': 3, 'gamma': 4.480579447590184}. Best is trial 19 with value: 1.2590178473485025.


Best trial: 19. Best value: 1.25902: 100%|██████████| 30/30 [10:43<00:00, 21.44s/it]

[I 2025-12-16 23:55:13,178] Trial 29 finished with value: 1.2974949120544943 and parameters: {'n_estimators': 384, 'learning_rate': 0.06005045592070487, 'max_depth': 8, 'subsample': 0.8003137070747316, 'colsample_bytree': 0.964459114629968, 'min_child_weight': 2, 'gamma': 2.2012199008004876}. Best is trial 19 with value: 1.2590178473485025.

✓ XGBoost HPO 완료!
최적 RMSLE: 1.259018
최적 하이퍼파라미터:
  n_estimators: 470
  learning_rate: 0.07337466316464045
  max_depth: 10
  subsample: 0.7101507633412831
  colsample_bytree: 0.8321208143455443
  min_child_weight: 4
  gamma: 4.146232569244199


### 📚 XGBoost HPO 설명
- **Optuna**: 베이지안 최적화 기반의 하이퍼파라미터 튜닝 라이브러리
- **탐색 범위**:
  - `n_estimators`: 100~800 (더 많은 트리)
  - `learning_rate`: 0.01~0.3 (학습률)
  - `max_depth`: 3~10 (트리 깊이)
  - `subsample/colsample_bytree`: 0.5~1.0 (샘플링 비율)
  - `min_child_weight`, `gamma`: 정규화 강도
- **TPE Sampler**: Tree-structured Parzen Estimator (효율적인 탐색)
- **n_trials=30**: 30번의 시도로 최적값 찾기 (더 높이려면 50, 100 시도 가능)

## 7️⃣-2 최적화된 XGBoost 모델 훈련

In [14]:
# 최적화된 XGBoost 모델 훈련
xgb_model = xgb.XGBRegressor(**xgb_best_params)

print("최적화된 XGBoost 모델 훈련 중...")
xgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# 검증 세트에서 예측
xgb_pred_val = xgb_model.predict(X_val)
xgb_metrics = calculate_metrics(y_val, xgb_pred_val, "XGBoost (Optimized)")

print(f"\n✓ HPO 전 vs 후 비교:")
print(f"  기본 파라미터 대비 개선도 계산 중...")

최적화된 XGBoost 모델 훈련 중...
XGBoost (Optimized) 평가 지표:
  RMSE:  415.8387
  RMSLE: 1.2590 (Kaggle 점수)
  MAE:   103.2684
  MAPE:  25220853628880760.0000

✓ HPO 전 vs 후 비교:
  기본 파라미터 대비 개선도 계산 중...
XGBoost (Optimized) 평가 지표:
  RMSE:  415.8387
  RMSLE: 1.2590 (Kaggle 점수)
  MAE:   103.2684
  MAPE:  25220853628880760.0000

✓ HPO 전 vs 후 비교:
  기본 파라미터 대비 개선도 계산 중...


In [15]:
# XGBoost 하이퍼파라미터
xgb_params = {
    'n_estimators': 500,       # 부스팅 라운드 수
    'learning_rate': 0.05,     # 각 부스팅 단계의 학습률 (낮을수록 안정적)
    'max_depth': 7,            # 트리의 최대 깊이 (과적합 방지)
    'subsample': 0.8,          # 각 트리에 사용할 샘플의 비율
    'colsample_bytree': 0.8,   # 각 트리에 사용할 특성의 비율
    'random_state': 42,
    'eval_metric': 'rmse',
    'verbose': 0,
    'device': 'cuda',           # ← GPU 활성화
    'tree_method': 'hist'       # ← GPU용 트리 방식
}

xgb_model = xgb.XGBRegressor(**xgb_params)

# 모델 훈련
print("XGBoost 모델 훈련 중...")
xgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# 검증 세트에서 예측
xgb_pred_val = xgb_model.predict(X_val)
xgb_metrics = calculate_metrics(y_val, xgb_pred_val, "XGBoost")

XGBoost 모델 훈련 중...
XGBoost 평가 지표:
  RMSE:  426.7756
  RMSLE: 1.3605 (Kaggle 점수)
  MAE:   91.8065
  MAPE:  23085940527292724.0000
XGBoost 평가 지표:
  RMSE:  426.7756
  RMSLE: 1.3605 (Kaggle 점수)
  MAE:   91.8065
  MAPE:  23085940527292724.0000


### 📚 설명
- **XGBoost**: eXtreme Gradient Boosting, 시계열 및 정형 데이터 최적화된 알고리즘
- **주요 하이퍼파라미터**:
  - **n_estimators=500**: 500개 트리 순차 구성 (더 많을수록 정확하나 느림)
  - **learning_rate=0.05**: 각 단계에서 5% 가중치로 학습 (작을수록 안정적, 크면 빠름)
  - **max_depth=7**: 트리 최대 깊이로 복잡도 제어
  - **subsample=0.8**: 샘플링으로 과적합 방지
  - **colsample_bytree=0.8**: 특성 샘플링으로 다양성 확보
- **eval_set**: 검증 세트로 훈련 진행도 모니터링
- **시계열 데이터에 강력**: 자동으로 특성 간 비선형 관계 학습

## 8️⃣ 모델 성능 비교 및 최종 예측

In [20]:
print("="*70)
print("📊 Store Sales Forecasting - XGBoost 최종 결과")
print("="*70)

print("\n✅ 데이터:")
print(f"  - 훈련 세트: {X_train_split.shape[0]:,} 샘플 (90개 특성)")
print(f"  - 검증 세트: {X_val.shape[0]:,} 샘플")
print(f"  - 테스트 세트: {X_test.shape[0]:,} 샘플")

print("\n✅ XGBoost 모델 성능:")
print(f"  - RMSLE: {xgb_metrics['RMSLE']:.4f}")
print(f"  - RMSE:  {xgb_metrics['RMSE']:.4f}")
print(f"  - MAE:   {xgb_metrics['MAE']:.4f}")
print(f"  - MAPE:  {xgb_metrics['MAPE']:.4f}")

print("\n✅ 예측값 통계:")
pred_stats = pd.Series(xgb_pred_test)
print(f"  - 최소값: {pred_stats.min():.2f}")
print(f"  - 평균값: {pred_stats.mean():.2f}")
print(f"  - 중앙값: {pred_stats.median():.2f}")
print(f"  - 최대값: {pred_stats.max():.2f}")

print("\n✅ 생성된 제출 파일:")
print(f"  - submission_final.csv")
print(f"  - 크기: {submission.shape[0]} 행 × {submission.shape[1]} 열")

print("\n" + "="*70)
print("🎯 다음 단계:")
print("  1. submission_final.csv를 Kaggle에 업로드")
print("  2. 리더보드에서 최종 점수 확인")
print("="*70)

📊 Store Sales Forecasting - XGBoost 최종 결과

✅ 데이터:
  - 훈련 세트: 2,406,412 샘플 (90개 특성)
  - 검증 세트: 601,604 샘플
  - 테스트 세트: 28,512 샘플

✅ XGBoost 모델 성능:
  - RMSLE: 1.3605
  - RMSE:  426.7756
  - MAE:   91.8065
  - MAPE:  23085940527292724.0000

✅ 예측값 통계:
  - 최소값: 0.00
  - 평균값: 260.45
  - 중앙값: 20.71
  - 최대값: 21762.98

✅ 생성된 제출 파일:
  - submission_final.csv
  - 크기: 28512 행 × 2 열

🎯 다음 단계:
  1. submission_final.csv를 Kaggle에 업로드
  2. 리더보드에서 최종 점수 확인


## 9️⃣ 최종 요약

### 📚 설명
- **Kaggle 제출 형식**: 두 개의 컬럼만 필요
  - **id**: store_nbr_family_date (고유 식별자)
  - **sales**: XGBoost로 예측된 판매액
- **제출 파일 검증**: 행 수 약 28K, 열 수 2개, 음수 값 없음

In [19]:
# Kaggle 요구 형식: id, sales
# test_meta의 id를 그냥 사용 (이미 제대로 된 형식)

submission = pd.DataFrame({
    'id': test_meta['id'].values,
    'sales': xgb_pred_test
})

# 제출 파일 저장
submission.to_csv('submission_final.csv', index=False)

print(f"✓ 제출 파일 생성 완료: submission_final.csv")
print(f"제출 파일 형태: {submission.shape}")
print(f"\n처음 10개 행:")
print(submission.head(10))

print(f"\nID 형식 예시: {submission['id'].iloc[0]}")
print(f"예측 판매액 범위: [{submission['sales'].min():.2f}, {submission['sales'].max():.2f}]")

✓ 제출 파일 생성 완료: submission_final.csv
제출 파일 형태: (28512, 2)

처음 10개 행:
        id       sales
0  3000888    0.000000
1  3000889    0.000000
2  3000890    0.000000
3  3000891  577.365112
4  3000892    0.000000
5  3000893  246.853455
6  3000894    2.809296
7  3000895  468.281616
8  3000896  351.227570
9  3000897  109.799652

ID 형식 예시: 3000888
예측 판매액 범위: [0.00, 21762.98]


## 8️⃣-2 Kaggle 제출 형식 생성

### 📚 설명
- **음수 클리핑**: 판매량은 음수가 불가능하므로, 0 미만의 예측값을 0으로 조정
- **예측값 범위 검토**: 일반적으로 0-8,500 범위 (훈련 데이터와 유사)

In [16]:
# 테스트 세트에서 최종 예측 생성
xgb_pred_test = xgb_model.predict(X_test)

# 음수 값 클리핑 (판매량은 0 이상)
xgb_pred_test = np.maximum(xgb_pred_test, 0)

print(f"XGBoost 예측 범위: [{xgb_pred_test.min():.2f}, {xgb_pred_test.max():.2f}]")
print(f"\n예측값 통계:")
print(pd.Series(xgb_pred_test).describe())

XGBoost 예측 범위: [0.00, 21762.98]

예측값 통계:
count    28512.000000
mean       260.451843
std        682.706848
min          0.000000
25%          3.434059
50%         20.711609
75%        170.944336
max      21762.982422
dtype: float64
